In [2]:
import GPflow
import GPy
import tensorflow as tf
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
plt.style.use('ggplot')
%matplotlib inline

import datasets
import category_dae
import missForest as mf

In [3]:
dataset = datasets.datasets(exclude_labels=False)["Soybean"]
info = category_dae.dataset_dimensions_info(dataset)
test_df, _ = mf.preprocess_dataframe(
    dataset[0], info, ignore_ordered=True)
test_df = test_df.dropna()
y = test_df[info["cat_dummies"]["Class"]]
X = test_df.drop(y.keys(), axis=1)


+++ Importing LetterRecognition
+++ Importing Shuttle
+++ Importing BreastCancer
+++ Importing BostonHousing
+++ Importing Servo
+++ Importing Ionosphere
V2 ; must have more than 1 possible value
+++ Importing Soybean


In [33]:
y = np.argmax(y.values, axis=1)[:, np.newaxis]
X = X.values
n_classes = y.max()+1
n_features = X.shape[1]
test_mask = np.random.rand(len(X)) < 0.2
X_train = X[~test_mask]
X_test = X[test_mask]
y_train = y[~test_mask]
y_test = y[test_mask]

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [46]:
kern = GPflow.kernels.Matern32(n_features, ARD=True) + GPflow.kernels.White(n_features, variance=0.01)
Z = X_train.copy()
np.random.shuffle(Z)
m = GPflow.svgp.SVGP(X_train, y_train, kern=kern,
                   likelihood=GPflow.likelihoods.MultiClass(n_classes),
                   Z=Z[:200], num_latent=n_classes)

In [47]:
%%time
m.optimize(maxiter=100)

CPU times: user 1min 29s, sys: 13.3 s, total: 1min 42s
Wall time: 55.5 s


      fun: 481.87423858304123
 hess_inv: <405601x405601 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -1.11233090e-03,   1.25575613e-03,  -1.27346927e-03, ...,
         2.84449215e-05,   2.67499044e-06,  -4.28594282e-03])
  message: b'STOP: TOTAL NO. of ITERATIONS EXCEEDS LIMIT'
     nfev: 115
      nit: 101
   status: 1
  success: False
        x: array([ -2.37872730e-01,   1.33053024e+00,  -2.34183133e-01, ...,
         2.68789244e-05,   2.96099576e-05,   9.93690877e-01])

In [44]:
print(accuracy_score(np.argmax(m.predict_y(X_train)[0], axis=1), y_train.flatten()))

0.942184154176


In [45]:
print(accuracy_score(np.argmax(m.predict_y(X_test)[0], axis=1), y_test.flatten()))

0.905263157895


In [9]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1,
                           max_features=int(np.floor(X_train.shape[1]**.5)),
                           bootstrap=False,
                           min_samples_split=5)
rf.fit(X_train, y_train)
train_perf = rf.score(X_train, y_train)
test_perf = rf.score(X_test, y_test)
print(accuracy_score(rf.predict(X_train), y_train))
print(accuracy_score(rf.predict(X_test), y_test))
print("Test: {:f}, Training: {:f}".format(test_perf, train_perf))

 /home/adria/venv/lib/python3.5/site-packages/ipykernel_launcher.py:5: DataConversionWarning:A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


0.997858672377
0.905263157895
Test: 0.905263, Training: 0.997859
